# Regression with neural network in pyTorch
### Training data

In [1]:
import numpy as np
import torch as tr

# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

### Convert the input target to tensor

In [2]:
input_data = tr.from_numpy(inputs)
target_data = tr.from_numpy(targets)
print(input_data)
print(target_data)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


### Weights and bias
``
Weights and bias initialized as random values. The first row of w and b are used to predict the first target variable. i.e., yield of apples and similarly the second for oranges
``

In [3]:
tr.random.manual_seed(42)
w = tr.randn(2, 3, requires_grad=True) # weights, need to grad is True
b = tr.randn(2, requires_grad=True) # bias, need to grad is True
print(w)
print(b)

tensor([[ 0.3367,  0.1288,  0.2345],
        [ 0.2303, -1.1229, -0.1863]], requires_grad=True)
tensor([ 2.2082, -0.6380], requires_grad=True)


### Define the model

In [4]:
def model(x):
    return x @ w.t() + b

In [5]:
### generate predictions

In [6]:
preds = model(input_data)
print(preds)

tensor([[  45.4987,  -67.0672],
        [  59.1878,  -90.4141],
        [  62.3595, -141.8688],
        [  50.7645,  -32.3210],
        [  54.2179, -105.5822]], grad_fn=<AddBackward0>)


In [7]:
# Compare the targets
print(target_data)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [8]:
# Loss function
### MSE loss

In [9]:
def mse(m, n):
    diff = m - n
    return tr.sum(diff*diff)/diff.numel()
# compute loss
loss = mse(preds, target_data)
print(loss)

tensor(19322.3477, grad_fn=<DivBackward0>)


In [10]:
### Compute gradiants

In [11]:
# compute the gradiants
loss.backward()

In [12]:
# gradiants for weights
print(w)
print(w.grad)

tensor([[ 0.3367,  0.1288,  0.2345],
        [ 0.2303, -1.1229, -0.1863]], requires_grad=True)
tensor([[ -1622.2405,  -2731.8169,  -1496.6278],
        [-14781.0176, -17480.2129, -10474.4824]])


In [13]:
w.grad

tensor([[ -1622.2405,  -2731.8169,  -1496.6278],
        [-14781.0176, -17480.2129, -10474.4824]])

In [14]:
with tr.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [15]:
# let's verify that the loss is actually lower
loss = mse(preds, target_data)
print(loss)

tensor(19322.3477, grad_fn=<DivBackward0>)


In [16]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [17]:
### Train the model using gradient descent

In [19]:
# Generate predictions
preds = model(input_data)
print(preds)

tensor([[ 49.1570, -40.0595],
        [ 64.0261, -54.8753],
        [ 68.2998, -99.5089],
        [ 54.1479,  -5.8505],
        [ 59.0077, -71.2684]], grad_fn=<AddBackward0>)


In [21]:
# Calculate the loss
loss = mse(preds, target_data)
print(loss)

tensor(13438.3047, grad_fn=<DivBackward0>)


In [22]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ -1242.2932,  -2317.3828,  -1242.2361],
        [-11989.3066, -14471.1055,  -8619.6689]])
tensor([ -17.2723, -146.3125])


In [24]:
# Adjust weights & reset gradients
with tr.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [25]:
print(w)
print(b)

tensor([[ 0.3653,  0.1793,  0.2619],
        [ 0.4980, -0.8033,  0.0046]], requires_grad=True)
tensor([ 2.2086, -0.6347], requires_grad=True)


In [26]:
# Calculate loss
preds = model(input_data)
loss = mse(preds, target_data)
print(loss)

tensor(9467.9111, grad_fn=<DivBackward0>)


In [27]:
## Train for multiple epochs

In [33]:
# train for 100 epochs
for i in range(150):
    preds = model(input_data)
    loss = mse(preds, target_data)
    
    loss.backward()
    with tr.no_grad():
        w -= w.grad*1e-5
        b -= b.grad*1e-5
        w.grad.zero_()
        b.grad.zero_()

In [34]:
# Calculate loss
preds = model(input_data)
loss = mse(preds, target_data)
print(loss)

tensor(62.1724, grad_fn=<DivBackward0>)


In [35]:
preds

tensor([[ 59.4326,  73.2340],
        [ 81.6480, 100.5570],
        [116.3258, 128.5114],
        [ 31.9058,  54.4224],
        [ 95.0663, 108.6577]], grad_fn=<AddBackward0>)

In [36]:
target_data

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])